In [5]:
import os
import glob
import ROOT as r
from collections import Counter
import math
import numpy as np
import random
from scipy.cluster.hierarchy import fclusterdata

In [7]:
# Set treename to either 'ST-b2oc' or 'ST-b2cc'
treenames = ["ST-b2cc"]

# Example bad run list
bad_runs = [303681, 304475, 304487, 304488, 304489, 304490, 304491]


for treename in treenames:
    # Determine which directory to check based on treename
    if treename == "ST-b2oc":
        data_dir = "./Processed_Data/2024/Real/B2OC"
    elif treename == "ST-b2cc":
        data_dir = "./Processed_Data/2024/Real/B2CC"
    else:
        raise ValueError("Unknown treename. Use 'ST-b2oc' or 'ST-b2cc'.")

    # Find all .root files in all subdirectories
    file_list = glob.glob(os.path.join(data_dir, "**", "*.root"), recursive=True)

    for filename in file_list:
        file = r.TFile.Open(filename)
        if not file or file.IsZombie():
            print(f"⚠️ Failed to open file: {filename}")
            continue
        tree = file.Get(treename)
        if not tree:
            print(f"Tree {treename} not found in {filename}")
            file.Close()
            continue

        # Check number of entries (should be > 20000)
        nm_entries = tree.GetEntries()
        if nm_entries < 20000:
            print(f"❌ For {filename}, number of entries = {nm_entries}: bad.")

        # Check if any bad runs
        for bad_run in bad_runs:
            n = tree.GetEntries(f"run == {bad_run}")
            if n > 0:
                print(f"❌ For {filename} the number of entries with run {bad_run}: {n}")

        # Check if unique block
        block_values = set()
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            block_val = tree.block
            if block_val is None or (isinstance(block_val, float) and math.isnan(block_val)):
                continue
            block_values.add(int(block_val))
            if len(block_values) > 1:
                print(f"❌ For {filename} the wrong blocks were found {block_values}")
                break

        # Check if unique fill
        fill = True
        if fill:
            fill_values = set()
            for i in range(tree.GetEntries()):
                tree.GetEntry(i)
                fill_val = tree.FILL
                if fill_val is None or (isinstance(fill_val, float) and math.isnan(fill_val)):
                    continue
                fill_values.add(int(fill_val))
                if len(fill_values) > 1:
                    print(f"❌ For {filename} the wrong fill values were found {fill_values}")
                    break

        # Check duplicates
        event_ids = []
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            event_ids.append(tree.event)
        counts = Counter(event_ids)
        num_duplicates = sum(count for count in counts.values() if count > 1)
        fraction_duplicates = num_duplicates / tree.GetEntries() if tree.GetEntries() > 0 else None
        print(f"For {filename}, number of duplicates: {num_duplicates} ({fraction_duplicates:.5%})")

        # Check for NaN or invalid values
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            if (
                not isinstance(tree.event, (int, float)) or (isinstance(tree.event, float) and math.isnan(tree.event)) or
                not isinstance(tree.Bp_DTF_OwnPV_MASS, (int, float)) or (isinstance(tree.Bp_DTF_OwnPV_MASS, float) and math.isnan(tree.Bp_DTF_OwnPV_MASS)) or
                not isinstance(tree.FILL, (int, float)) or (isinstance(tree.FILL, float) and math.isnan(tree.FILL)) or
                not isinstance(tree.run, (int, float)) or (isinstance(tree.run, float) and math.isnan(tree.run))
            ):
                print(f"❌ For file name {filename}, event {tree.event}, Bp_DTF_OwnPV_MASS {tree.Bp_DTF_OwnPV_MASS}, FILL {tree.FILL}, run {tree.run} has NaN or non-numeric values.")
                break
        file.Close()

For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10061.root, number of duplicates: 302100 (28.14909%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10077.root, number of duplicates: 523475 (27.74014%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10098.root, number of duplicates: 282324 (27.68797%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10082.root, number of duplicates: 140399 (27.94583%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10094.root, number of duplicates: 200042 (27.83112%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10095.root, number of duplicates: 789182 (27.65135%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10099.root, number of duplicates: 329339 (27.78114%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10084.root, number of duplicates: 1293536 (27.94849%)
For ./Processed_Data/2024/Real/B2CC/B5/2024_B2CC_UP_B5_F10092.root, number of duplicates: 830784 (27.76829%)
For ./Processed_Da

In [9]:
# Set treename to either 'ST-b2oc' or 'ST-b2cc'
treenames = ["ST-b2oc", "ST-b2cc"]

# Example bad run list
bad_runs = [303681, 304475, 304487, 304488, 304489, 304490, 304491]


for treename in treenames:
    # Determine which directory to check based on treename
    if treename == "ST-b2oc":
        data_dir = "./Processed_Data/2024/Real/B2OC"
    elif treename == "ST-b2cc":
        data_dir = "./Processed_Data/2024/Real/B2CC"
    else:
        raise ValueError("Unknown treename. Use 'ST-b2oc' or 'ST-b2cc'.")

    # Find all .root files that directory, without checking subdirectories
    file_list = glob.glob(os.path.join(data_dir, "*.root"))

    for filename in file_list:
        file = r.TFile.Open(filename)
        if not file or file.IsZombie():
            print(f"⚠️ Failed to open file: {filename}")
            continue
        tree = file.Get(treename)
        if not tree:
            print(f"Tree {treename} not found in {filename}")
            file.Close()
            continue

        # Check number of entries (should be > 20000)
        nm_entries = tree.GetEntries()
        if nm_entries < 20000:
            print(f"❌ For {filename}, number of entries = {nm_entries}: bad.")

        # Check if any bad runs
        for bad_run in bad_runs:
            n = tree.GetEntries(f"run == {bad_run}")
            if n > 0:
                print(f"❌ For {filename} the number of entries with run {bad_run}: {n}")

        # Check if unique block
        block_values = set()
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            block_val = tree.block
            if block_val is None or (isinstance(block_val, float) and math.isnan(block_val)):
                continue
            block_values.add(int(block_val))
            if len(block_values) > 1:
                print(f"❌ For {filename} the wrong blocks were found {block_values}")
                break

        # Check duplicates
        event_ids = []
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            event_ids.append(tree.event)
        counts = Counter(event_ids)
        num_duplicates = sum(count for count in counts.values() if count > 1)
        fraction_duplicates = num_duplicates / tree.GetEntries() if tree.GetEntries() > 0 else None
        print(f"For {filename}, number of duplicates: {num_duplicates} ({fraction_duplicates:.5%})")

        # Check for NaN or invalid values
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            if (
                not isinstance(tree.event, (int, float)) or (isinstance(tree.event, float) and math.isnan(tree.event)) or
                not isinstance(tree.Bp_DTF_OwnPV_MASS, (int, float)) or (isinstance(tree.Bp_DTF_OwnPV_MASS, float) and math.isnan(tree.Bp_DTF_OwnPV_MASS)) or
                not isinstance(tree.FILL, (int, float)) or (isinstance(tree.FILL, float) and math.isnan(tree.FILL)) or
                not isinstance(tree.run, (int, float)) or (isinstance(tree.run, float) and math.isnan(tree.run))
            ):
                print(f"❌ For file name {filename}, event {tree.event}, Bp_DTF_OwnPV_MASS {tree.Bp_DTF_OwnPV_MASS}, FILL {tree.FILL}, run {tree.run} has NaN or non-numeric values.")
                break
        file.Close()

For ./Processed_Data/2024/Real/B2OC/2024_B2OC_DOWN_B6.root, number of duplicates: 401583 (3.95837%)
For ./Processed_Data/2024/Real/B2OC/2024_B2OC_UP_B5.root, number of duplicates: 43716 (4.05792%)
For ./Processed_Data/2024/Real/B2OC/2024_B2OC_UP_B8.root, number of duplicates: 288437 (4.94885%)
For ./Processed_Data/2024/Real/B2OC/2024_B2OC_DOWN_B7.root, number of duplicates: 445794 (4.75272%)
For ./Processed_Data/2024/Real/B2CC/2024_B2CC_UP_B8.root, number of duplicates: 7708312 (30.91635%)
For ./Processed_Data/2024/Real/B2CC/2024_B2CC_UP_B5.root, number of duplicates: 12061753 (27.83886%)
For ./Processed_Data/2024/Real/B2CC/2024_B2CC_DOWN_B7.root, number of duplicates: 12643616 (30.77410%)
For ./Processed_Data/2024/Real/B2CC/2024_B2CC_DOWN_B6.root, number of duplicates: 9877840 (27.56504%)


Warning in <TFile::Init>: file ./Processed_Data/2024/Real/B2OC/2024_B2OC_UP_B5.root probably not closed, trying to recover
Info in <TFile::Recover>: ./Processed_Data/2024/Real/B2OC/2024_B2OC_UP_B5.root, recovered key TTree:ST-b2oc at address 151520678
Warning in <TFile::Init>: successfully recovered 1 keys


In [ ]:
filename = ["./Data/00289237_00000001_1.highstats-Small-B2OC-UP.root"]

bad_runs = [303681, 304475, 304487, 304488, 304489, 304490, 304491]

tree_name = "ST-b2oc"

tree = r.TChain(tree_name)
for fname in filename:
    tree.Add(fname)

if not tree:
    print(f"Tree {tree_name} not found in {filename}")
else:
    fill_counts = {}
    for i in range(tree.GetEntries()):
        tree.GetEntry(i)
        block_val = tree.block
        run_val = tree.run
        fill_val = tree.FILL
        # Filter: block not None/NaN, not 0 or -1; fill not None/NaN; run not None/NaN and not in bad_runs
        if block_val is None or (isinstance(block_val, float) and math.isnan(block_val)):
            continue
        block_val = int(block_val)
        if block_val in [0, -1]:
            continue
        if fill_val is None or (isinstance(fill_val, float) and math.isnan(fill_val)):
            continue
        if run_val is None or (isinstance(run_val, float) and math.isnan(run_val)):
            continue
        run_val = int(run_val)
        if run_val in bad_runs:
            continue
        fill_val = int(fill_val)
        fill_counts[fill_val] = fill_counts.get(fill_val, 0) + 1
    print("Count per unique fill (after filtering):")
    for fill, count in sorted(fill_counts.items()):
        print(f"FILL {fill}: {count} entries")

In [10]:
file = "./Data/00289229_0000000*_1.highstats-Small-B2CC-UP.root"
tree = r.TChain("ST-b2cc")
tree.Add(file)
tree.Print()

******************************************************************************
*Chain   :ST-b2cc   : ./Data/00289229_00000001_1.highstats-Small-B2CC-UP.root *
******************************************************************************
******************************************************************************
*Tree    :ST-b2cc   : Small Tree                                             *
*Entries : 16507786 : Total =      3037883223 bytes  File  Size = 2102479648 *
*        :          : Tree compression factor =   1.44                       *
******************************************************************************
*Br    0 :event     : event/l                                                *
*Entries : 16507786 : Total  Size=  132074761 bytes  File Size  =   79224549 *
*Baskets :      128 : Basket Size=    1920000 bytes  Compression=   1.67     *
*............................................................................*
*Br    1 :run       : run/L                        

In [ ]:
def clean_fill_files_by_event_and_momentum_greedy():
    # Directories
    base_in = "./Processed_Data/2024/Real"
    base_out = "./Processed_Data_Clean/2024/Real"
    folders = ["B2CC", "B2OC"]
    treenames = {"B2CC": "ST-b2cc", "B2OC": "ST-b2oc"}
    import random

    for folder in folders:
        in_dir = os.path.join(base_in, folder)
        out_dir = os.path.join(base_out, folder)
        # Find all fill files recursively
        file_list = glob.glob(os.path.join(in_dir, "**", "*.root"), recursive=True)
        for infile in file_list:
            # Determine output path
            rel_path = os.path.relpath(infile, in_dir)
            outfile = os.path.join(out_dir, rel_path)
            os.makedirs(os.path.dirname(outfile), exist_ok=True)
            treename = treenames[folder]
            file = r.TFile.Open(infile)
            if not file or file.IsZombie():
                print(f"⚠️ Failed to open file: {infile}")
                continue
            tree = file.Get(treename)
            if not tree:
                print(f"Tree {treename} not found in {infile}")
                file.Close()
                continue
            # Collect all entries: (event, Bp_P, entry index)
            events = {}
            for i in range(tree.GetEntries()):
                tree.GetEntry(i)
                event = tree.event
                bp_p = tree.Bp_P
                if event is None or bp_p is None:
                    continue
                if event not in events:
                    events[event] = []
                events[event].append((bp_p, i))
            # For each event, apply greedy ±5% deduplication
            keep_indices = set()
            for event, bp_p_list in events.items():
                # Sort by Bp_P for deterministic behavior
                bp_p_list = sorted(bp_p_list, key=lambda x: x[0]) # This line sorts by Bp_P
                used = [False] * len(bp_p_list) 
                for idx, (bp_p, entry_idx) in enumerate(bp_p_list):
                    if used[idx]:
                        continue
                    # Find all within ±5% of bp_p
                    group = [j for j, (other_p, _) in enumerate(bp_p_list)
                             if not used[j] and abs(other_p - bp_p) <= 0.05 * bp_p]
                    # Randomly select one to keep
                    chosen = random.choice(group)
                    keep_indices.add(bp_p_list[chosen][1])
                    # Mark all in group as used
                    for j in group:
                        used[j] = True
            # Write filtered entries to new file
            out_file = r.TFile(outfile, "RECREATE")
            new_tree = tree.CloneTree(0)
            for i in range(tree.GetEntries()):
                if i in keep_indices:
                    tree.GetEntry(i)
                    new_tree.Fill()
            new_tree.Write()
            out_file.Close()
            file.Close()
            print(f"✅ Cleaned and saved: {outfile} ({len(keep_indices)} entries)")

In [ ]:
def clean_fill_files_by_event_and_momentum_overlap():
    """
    For each event, group all entries whose ±5% Bp_P intervals overlap (directly or indirectly),
    and keep only one entry per group. This avoids bias from sequential greedy selection.
    """
    import random
    base_in = "./Processed_Data/2024/Real"
    base_out = "./Processed_Data_Clean/2024/Real"
    folders = ["B2CC", "B2OC"]
    treenames = {"B2CC": "ST-b2cc", "B2OC": "ST-b2oc"}

    def find_overlap_groups(bp_p_list):
        # Build graph: edge if intervals overlap
        n = len(bp_p_list)
        adj = [[] for _ in range(n)]
        for i in range(n):
            bp_p_i = bp_p_list[i][0]
            for j in range(i+1, n):
                bp_p_j = bp_p_list[j][0]
                # Intervals overlap if abs(bp_p_i - bp_p_j) <= 0.05*bp_p_i or 0.05*bp_p_j
                # But more generally, intervals [bp_p*(0.95), bp_p*(1.05)] overlap if they intersect
                low_i, high_i = bp_p_i*0.95, bp_p_i*1.05
                low_j, high_j = bp_p_j*0.95, bp_p_j*1.05
                if not (high_i < low_j or high_j < low_i):
                    adj[i].append(j)
                    adj[j].append(i)
        # Find connected components (BFS)
        groups = []
        visited = [False]*n
        for i in range(n):
            if not visited[i]:
                queue = [i]
                group = []
                visited[i] = True
                while queue:
                    node = queue.pop()
                    group.append(node)
                    for neighbor in adj[node]:
                        if not visited[neighbor]:
                            visited[neighbor] = True
                            queue.append(neighbor)
                groups.append(group)
        return groups

    for folder in folders:
        in_dir = os.path.join(base_in, folder)
        out_dir = os.path.join(base_out, folder)
        file_list = glob.glob(os.path.join(in_dir, "**", "*.root"), recursive=True)
        for infile in file_list:
            rel_path = os.path.relpath(infile, in_dir)
            outfile = os.path.join(out_dir, rel_path)
            os.makedirs(os.path.dirname(outfile), exist_ok=True)
            treename = treenames[folder]
            file = r.TFile.Open(infile)
            if not file or file.IsZombie():
                print(f"⚠️ Failed to open file: {infile}")
                continue
            tree = file.Get(treename)
            if not tree:
                print(f"Tree {treename} not found in {infile}")
                file.Close()
                continue
            events = {}
            for i in range(tree.GetEntries()):
                tree.GetEntry(i)
                event = tree.event
                bp_p = tree.Bp_P
                if event is None or bp_p is None:
                    continue
                if event not in events:
                    events[event] = []
                events[event].append((bp_p, i))
            keep_indices = set()
            for event, bp_p_list in events.items():
                groups = find_overlap_groups(bp_p_list)
                for group in groups:
                    chosen = random.choice(group)
                    keep_indices.add(bp_p_list[chosen][1])
            out_file = r.TFile(outfile, "RECREATE")
            new_tree = tree.CloneTree(0)
            for i in range(tree.GetEntries()):
                if i in keep_indices:
                    tree.GetEntry(i)
                    new_tree.Fill()
            new_tree.Write()
            out_file.Close()
            file.Close()
            print(f"✅ Cleaned and saved: {outfile} ({len(keep_indices)} entries)")

In [ ]:
base_in = "./Processed_Data/2024/Real"
base_out = "./Processed_Data_Clean_Bp_P/2024/Real"
folders = ["B2CC", "B2OC"]
treenames = {"B2CC": "ST-b2cc", "B2OC": "ST-b2oc"}
epsilon=0.005

for folder in folders:
    in_dir = os.path.join(base_in, folder)
    out_dir = os.path.join(base_out, folder)
    file_list = glob.glob(os.path.join(in_dir, "**", "*.root"), recursive=True)
    for infile in file_list:
        rel_path = os.path.relpath(infile, in_dir)
        outfile = os.path.join(out_dir, rel_path)
        os.makedirs(os.path.dirname(outfile), exist_ok=True)
        treename = treenames[folder]
        file = r.TFile.Open(infile)
        if not file or file.IsZombie():
            print(f"⚠️ Failed to open file: {infile}")
            continue
        tree = file.Get(treename)
        if not tree:
            print(f"Tree {treename} not found in {infile}")
            file.Close()
            continue
        events = {}
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            event = tree.event
            bp_p = tree.Bp_P
            if event is None or bp_p is None:
                continue
            if event not in events:
                events[event] = []
            events[event].append((bp_p, i))
        print("✅ All events collected, now clustering...")
        keep_indices = set()
        for event, bp_p_list in events.items():
            if len(bp_p_list) == 1:
                keep_indices.add(bp_p_list[0][1])
                continue
            arr = np.array([[x[0]] for x in bp_p_list]) 
            clusters = fclusterdata(
                arr, t=epsilon, criterion='distance', method='single',
                metric=lambda u, v: abs(u[0] - v[0]) / min(u[0], v[0]) if min(u[0], v[0]) > 0 else np.inf
            )
            for cl in set(clusters):
                members = [i for i, c in enumerate(clusters) if c == cl]
                chosen = random.choice(members)
                keep_indices.add(bp_p_list[chosen][1])
        out_file = r.TFile(outfile, "RECREATE")
        new_tree = tree.CloneTree(0)
        for i in range(tree.GetEntries()):
            if i in keep_indices:
                tree.GetEntry(i)
                new_tree.Fill()
        new_tree.Write()
        out_file.Close()
        file.Close()
        print(f"✅ Cleaned and saved: {outfile} ({len(keep_indices)} entries)")

In [3]:
import random
from scipy.cluster.hierarchy import fclusterdata
import numpy as np
file_name = "test_duplicates.root"
outfile = "./test/test_duplicates_cleaned.root"
# Ensure output directory exists
os.makedirs(os.path.dirname(outfile), exist_ok=True)
treename = "ST-b2cc"
file = r.TFile.Open(file_name)
tree = file.Get(treename)
print(tree)
if not tree:
    print(f"Tree {treename} not found in {file_name}")
    file.Close()
# Collect all entries: (event, Bp_P, entry index)
events = {}
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    event = tree.event
    bp_p = tree.Bp_P
    if event is None or bp_p is None:
        continue
    if event not in events:
        events[event] = []
    events[event].append((bp_p, i))
# For each event, apply relative difference clustering deduplication
epsilon = 0.01  # 1% relative difference
keep_indices = set()
for event, bp_p_list in events.items():
    if len(bp_p_list) == 1:
        keep_indices.add(bp_p_list[0][1])
        continue
    arr = np.array([[x[0]] for x in bp_p_list])
    clusters = fclusterdata(
        arr, t=epsilon, criterion='distance', method='single',
        metric=lambda u, v: abs(u[0] - v[0]) / min(u[0], v[0]) if min(u[0], v[0]) > 0 else np.inf
    )
    for cl in set(clusters):
        members = [i for i, c in enumerate(clusters) if c == cl]
        chosen = random.choice(members)
        keep_indices.add(bp_p_list[chosen][1])
# Write filtered entries to new file
out_file = r.TFile(outfile, "RECREATE")
new_tree = tree.CloneTree(0)
for i in range(tree.GetEntries()):
    if i in keep_indices:
        tree.GetEntry(i)
        new_tree.Fill()
new_tree.Write()
out_file.Close()
file.Close()
print(f"✅ Cleaned and saved: {outfile} ({len(keep_indices)} entries)")

Name: ST-b2cc Title: Test Tree with duplicate events
✅ Cleaned and saved: ./test/test_duplicates_cleaned.root (8 entries)


In [8]:
file_name = "test_duplicates.root"
r.TFile.Open(file_name)
tree_name = "ST-b2cc"
tree = r.TChain(tree_name)
tree.Add(file_name)

for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    print(f"Event: {tree.event}, Bp_P: {tree.Bp_P}, Entry Index: {i}")

Event: 1121654010, Bp_P: 139387.8125, Entry Index: 0
Event: 1121654010, Bp_P: 135842.546875, Entry Index: 1
Event: 1121654010, Bp_P: 119004.640625, Entry Index: 2
Event: 1121654010, Bp_P: 111138.6875, Entry Index: 3
Event: 1121654010, Bp_P: 138274.65625, Entry Index: 4
Event: 19726357590, Bp_P: 127264.421875, Entry Index: 5
Event: 19726357590, Bp_P: 121160.2578125, Entry Index: 6
Event: 19726357590, Bp_P: 127050.9453125, Entry Index: 7
Event: 19726357590, Bp_P: 120945.6796875, Entry Index: 8
Event: 19726357590, Bp_P: 121916.0703125, Entry Index: 9
Event: 19726357590, Bp_P: 115807.109375, Entry Index: 10
Event: 19726357590, Bp_P: 121702.2890625, Entry Index: 11
Event: 19726357590, Bp_P: 115592.078125, Entry Index: 12
Event: 19726357590, Bp_P: 122707.90625, Entry Index: 13
Event: 19726357590, Bp_P: 122491.2265625, Entry Index: 14
Event: 19726357590, Bp_P: 117349.453125, Entry Index: 15
Event: 19726357590, Bp_P: 117132.59375, Entry Index: 16


In [11]:
file_name = "./test/test_duplicates_cleaned.root"
r.TFile.Open(file_name)
tree_name = "ST-b2cc"
tree = r.TChain(tree_name)
tree.Add(file_name)

for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    print(f"Event: {tree.event}, Bp_P: {tree.Bp_P}, Entry Index: {i}")

Event: 1121654010, Bp_P: 139387.8125, Entry Index: 0
Event: 1121654010, Bp_P: 135842.546875, Entry Index: 1
Event: 1121654010, Bp_P: 119004.640625, Entry Index: 2
Event: 1121654010, Bp_P: 111138.6875, Entry Index: 3
Event: 19726357590, Bp_P: 121160.2578125, Entry Index: 4
Event: 19726357590, Bp_P: 127050.9453125, Entry Index: 5
Event: 19726357590, Bp_P: 115807.109375, Entry Index: 6
Event: 19726357590, Bp_P: 117349.453125, Entry Index: 7
